In [1]:
%load_ext autoreload
%matplotlib widget
%autoreload 2
from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
    run_parallel_post,
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)
from time import time_ns
from amftrack.util.dbx import upload_folders, load_dbx, download, get_dropbox_folders
from datetime import datetime
from amftrack.pipeline.functions.image_processing.experiment_util import (
    plot_full_image_with_features,
)
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    load_graphs,
)
from amftrack.pipeline.functions.post_processing.extract_study_zone import (
    load_study_zone,
)
from amftrack.pipeline.functions.image_processing.experiment_util import (
    get_random_edge,
    distance_point_edge,
    plot_edge,
    plot_edge_cropped,
    find_nearest_edge,
    get_edge_from_node_labels,
    plot_full_image_with_features,
    get_all_edges,
    get_all_nodes,
    find_neighboring_edges,
    reconstruct_image,
    reconstruct_skeletton_from_edges,
    reconstruct_skeletton_unicolor,
    plot_edge_width,
    reconstruct_image_from_general,
    plot_full,
)
from amftrack.pipeline.functions.image_processing.extract_width_fun import (
    get_width_info,
    get_width_info_new,
)

/gpfs/home2/cbisot/pycode/AMFtrack/amftrack/util/dbx.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
2023-02-23 18:26:51.472309: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-23 18:26:51.586451: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /gpfs/home2/cbisot/miniconda3/envs/amftrack/lib/python3.7/site-packages/cv2/../../lib64:
2023-02-23 18:26:51.586484: I tensorflow/stream_executor/cuda/cudart_stub.cc:29

In [2]:
plates = [
    "078_20220920",
    "079_20220920",
    "080_20220920",
    "081_20220920",
    "082_20220920",
    "083_20220920",
    "084_20220920",
    "001_20220920",
    "024_20220920",
    "073_20220920",
    "002_20220920",
    "003_20220920",
    "004_20220920",
    "005_20220920",
    "006_20220920",
    "007_20220920",
    "008_20220920",
    "009_20220920",
    "010_20220920",
    "011_20220920",
    "012_20220920",
    "019_20220920",
    "020_20220920",
    "021_20220920",
    "022_20220920",
    "023_20220920",
    "037_20220920",
    "038_20220920",
    "039_20220920",
    "040_20220920",
    "042_20220920",
    "043_20220920",
    "044_20220920",
    "045_20220920",
    "046_20220920",
    "016_20220921",
    "017_20220921",
    "034_20220921",
    "051_20220920",
    "094_20220921",
    "064_20220921",
    "026_20220921",
    "027_20220921",
    "028_20220921",
    "029_20220921",
    "031_20220921",
    "033_20220921",
    "035_20220921",
    "036_20220921",
    "047_20220920",
    "041_20220920",
    "015_20220921",
    "068_20220921",
    "107_20220921",
    "095_20220921",
    "062_20220921",
    "096_20220921",
    "104_20220921",
    "050_20220920",
    "030_20220921",
    "025_20220921",
    "100_20220921",
    "067_20220921",
    "048_20220920",
    "055_20220921",
    "056_20220921",
    "057_20220921",
    "058_20220921",
    "059_20220921",
    "060_20220921",
    "061_20220921",
    "063_20220921",
    "014_20220921",
    "018_20220921",
    "099_20220921",
    "069_20220921",
    "054_20220920",
    "075_20220920",
    "092_20220921",
    "093_20220921",
    "097_20220921",
    "098_20220921",
    "105_20220921",
    "106_20220921",
    "108_20220921",
    "085_20220921",
    "086_20220921",
    "087_20220921",
    "088_20220921",
    "089_20220921",
    "090_20220921",
    "091_20220921",
    "049_20220920",
    "052_20220920",
    "053_20220920",
    "074_20220920",
    "076_20220920",
    "077_20220920",
    "013_20220921",
    "070_20220921",
    "071_20220921",
    "072_20220921",
    "102_20220921",
    "032_20220921",
    "101_20220921",
    "065_20220921",
    "066_20220921",
    "103_20220921",
]
plates = [
    "_".join((str(int(plate.split("_")[0])), str(int(plate.split("_")[1]))))
    for plate in plates
]
directory_targ = directory_project

In [43]:
plates = [
    # "1048_20220506",
    "32_20220428",
    "33_20220502",
    "52_20220707",
    "84_20220621",
    "91_20220708",
    "938_20220325",
    "1014_20220525",
    "1023_20220502",
    "1042_20220523",
    "1045_20220504",
    "1053_20220413",
    "1076_20220511",
]

In [2]:
plates = ["52_20220707"]

In [3]:
directory_targ = directory_project

In [6]:
update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[
    analysis_info["unique_id"].astype(str).isin(plates)
]

In [9]:
folders

,total_path,Plate,strain,medium,CrossDate,/Img/TileConfiguration.txt.registered,/Analysis/skeleton_compressed.mat,/Analysis/skeleton_masked_compressed.mat,/Analysis/skeleton_pruned_compressed.mat,/Analysis/transform.mat,...,/Analysis/nx_graph_pruned_labeled.p,date,folder,PrincePos,root,split,Pbait,Temperature,unique_id,datetime
0,/projects/0/einf914/data/20220920_1045_Plate01,077,'A5','GR24',20220920,True,True,True,True,False,...,True,"20.09.2022, 10:45:",20220920_1045_Plate01,NaN,NaN,NaN,NaN,NaN,77_20220920,2022-09-20 10:45:00
3,/projects/0/einf914/data/20220923_1410_Plate01,039,'A5','5DS',20220920,True,True,True,True,True,...,True,"23.09.2022, 14:10:",20220923_1410_Plate01,NaN,NaN,NaN,NaN,NaN,39_20220920,2022-09-23 14:10:00
4,/projects/0/einf914/data/20220926_1254_Plate01,006,'A5','control',20220920,True,True,True,True,True,...,True,"26.09.2022, 12:54:",20220926_1254_Plate01,NaN,NaN,NaN,NaN,NaN,6_20220920,2022-09-26 12:54:00
7,/projects/0/einf914/data/20220921_1717_Plate01,020,'A5-C2','control',20220920,True,True,True,True,True,...,True,"21.09.2022, 17:17:",20220921_1717_Plate01,NaN,NaN,NaN,NaN,NaN,20_20220920,2022-09-21 17:17:00
9,/projects/0/einf914/data/20221003_1817_Plate01,002,'A5','control',20220920,True,True,True,True,True,...,True,"03.10.2022, 18:17:",20221003_1817_Plate01,NaN,NaN,NaN,NaN,NaN,2_20220920,2022-10-03 18:17:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1323,/projects/0/einf914/data/20220923_1236_Plate01,107,'Giga-C2','GR24',20220921,True,True,True,True,False,...,False,"23.09.2022, 12:36:",20220923_1236_Plate01,NaN,NaN,NaN,NaN,NaN,107_20220921,2022-09-23 12:36:00
1324,/projects/0/einf914/data/20221003_1217_Plate01,044,'C2','5DS',20220920,True,True,True,True,True,...,True,"03.10.2022, 12:17:",20221003_1217_Plate01,NaN,NaN,NaN,NaN,NaN,44_20220920,2022-10-03 12:17:00
1328,/projects/0/einf914/data/20220930_1751_Plate01,066,'Giga-A5','5DS',20220921,True,True,True,True,False,...,True,"30.09.2022, 17:51:",20220930_1751_Plate01,NaN,NaN,NaN,NaN,NaN,66_20220921,2022-09-30 17:51:00
1332,/projects/0/einf914/data/20220923_1244_Plate01,086,'Giga','GR24',20220921,True,True,True,True,True,...,True,"23.09.2022, 12:44:",20220923_1244_Plate01,NaN,NaN,NaN,NaN,NaN,86_20220921,2022-09-23 12:44:00


In [4]:
update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)

analysed:   0%|          | 0/1932 [00:00<?, ?it/s]

In [5]:
# plates = ["1048_20220506"]
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]

In [7]:
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]

In [8]:
analysis_folders

,folder_analysis,version,total_path,Plate,strain,medium,CrossDate,date,unique_id,datetime,...,path_time_hypha_info,path_time_plate_info,path_global_plate_info,date_run_analysis,PrincePos,root,split,Pbait,Temperature,Temp
63,Analysis_52_20220707_200_Version1,Version1,/scratch-shared/amftrack/stitch_temp2/20220707...,52,'A5','001P100N100C',20220707,"07.07.2022, 14:58:",52_20220707,2022-07-07 14:58:00,...,Analysis_52_20220707_200_Version1/time_hypha_info,Analysis_52_20220707_200_Version1/time_plate_i...,Analysis_52_20220707_200_Version1/global_plate...,"01.01.1970, 01:00:",7,'Carrot','Y','-',25,nan


In [22]:
folders.columns

Index(['total_path', 'Plate', 'PrincePos', 'root', 'strain', 'medium', 'split',
       'Pbait', 'CrossDate', 'Temperature',
       '/Img/TileConfiguration.txt.registered',
       '/Analysis/skeleton_compressed.mat',
       '/Analysis/skeleton_masked_compressed.mat',
       '/Analysis/skeleton_pruned_compressed.mat', '/Analysis/transform.mat',
       '/Analysis/transform_corrupt.mat',
       '/Analysis/skeleton_realigned_compressed.mat',
       '/Analysis/nx_graph_pruned.p', '/Analysis/nx_graph_pruned_width.p',
       '/Analysis/nx_graph_pruned_labeled.p', 'date', 'folder', 'unique_id',
       'datetime'],
      dtype='object')

In [ ]:
time = "2:00:00"
name_job = "malin"

list_args = [[]] * len(list_f)
overwrite = True
num_parallel = 6
run_parallel_post(
    "time_edge_post_process.py",
    list_f,
    list_args,
    [directory, overwrite],
    analysis_folders,
    num_parallel,
    time,
    "edge_post_process",
    cpus=32,
    name_job=name_job,
    node="fat",
)

In [39]:
load_graphs(exp, directory_targ, indexes=[0], post_process=True)

TypeError: 'bool' object is not callable

In [21]:
path_save = (
    "/projects/0/einf914/data/20220521_0054_Plate03/Analysis/nx_graph_pruned_labeled2.p"
)

In [22]:
(g, pos) = pickle.load(open(path_save, "rb"))

EOFError: Ran out of input

In [36]:
import pickle

# row = [row for index, row in select.iterrows()][0]
path_exp = f'{directory_targ}{row["path_exp"]}'
exp = pickle.load(open(path_exp, "rb"))
exp.save_location = "/".join(path_exp.split("/")[:-1])

In [9]:
import pandas as pd

# list_args= [[500],[1000],[2000]]+[[]]
# list_args= [[500]]
overwrite = True
load_graphs = True
num_parallel = 20
time = "5:00:00"
for index, row in analysis_folders.iterrows():
    folder = row["folder_analysis"]
    path_time_plate_info = row["path_time_plate_info"]
    plate = row["Plate"]
    num_cpus = 32
    if os.path.isfile(f"{directory_targ}{path_time_plate_info}"):
        whole_plate_info = pd.read_json(
            f"{directory_targ}{path_time_plate_info}", convert_dates=True
        ).transpose()
        whole_plate_info.index.name = "t"
        whole_plate_info.reset_index(inplace=True)
        run_parallel_post(
            "recompute_width.py",
            [],
            [],
            [directory_targ, overwrite, load_graphs],
            whole_plate_info,
            num_parallel,
            time,
            "recompute_width",
            cpus=num_cpus,
            name_job="recompute2",
            node="fat",
        )

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 7680 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2278993


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 7680 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2278994


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 7680 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


...

Submitted batch job 2279001


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 7680 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2279002


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 7680 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.25 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2279003


In [31]:
row["path_exp"]

'Analysis_33_20220502_188_Version1/experiment.pick'

In [45]:
import pickle
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    load_graphs,
)

for index, row in whole_plate_info.iterrows():
    if index == 1:
        print(row)
        t = row["t"]
        path_exp = f'{directory_targ}{row["path_exp"]}'
        exp = pickle.load(open(path_exp, "rb"))
        exp.save_location = "/".join(path_exp.split("/")[:-1])
        try:
            exp.labeled
        except AttributeError:
            exp.labeled = True  # For older versions of experiments, to be removed later
        load_study_zone(exp)
        load_graphs(exp, directory_targ, indexes=[t], post_process=True)
        # load_skel(exp,[t])
        # print('size after loading',get_size(exp)/10**6)

        exp.load_tile_information(t)
        (G, pos) = exp.nx_graph[t], exp.positions[t]
        break

t                                                    1
num_trunks                                           3
area                                         841.36826
area_sep_comp                               517.863412
num_tips                                           803
                                               ...    
ring_biovolume_density_incr-100_index-15           0.0
ring_biovolume_density_incr-100_index-16           0.0
ring_biovolume_density_incr-100_index-17          None
ring_biovolume_density_incr-100_index-18          None
ring_biovolume_density_incr-100_index-19          None
Name: 1, Length: 95, dtype: object


FileNotFoundError: [Errno 2] No such file or directory: '/projects/0/einf914/data/20220504_2312_Plate07/Analysis/nx_graph_pruned_labeled2.p'

In [ ]:
name_job = "recompute"

list_f = [
    get_tot_length_C,
    get_tot_length_straight,
    get_time_since_start,
    get_time_since_begin_exp,
    get_width_edge,
    get_pos_x,
    get_pos_y,
]
list_args = [{}] * len(list_f)
# list_args= [[500],[1000],[2000]]+[[]]
# list_args= [[500]]
overwrite = True
load_graphs = True
num_parallel = 32
time = "3:00:00"
for index, row in analysis_folders.iterrows():
    folder = row["folder_analysis"]
    path_time_plate_info = row["path_time_plate_info"]
    plate = row["Plate"]
    num_cpus = 32
    if os.path.isfile(f"{directory_targ}{path_time_plate_info}"):
        whole_plate_info = pd.read_json(
            f"{directory_targ}{path_time_plate_info}", convert_dates=True
        ).transpose()
        whole_plate_info.index.name = "t"
        whole_plate_info.reset_index(inplace=True)
        run_parallel_post(
            "time_edge_post_process.py",
            list_f,
            list_args,
            [directory_targ, overwrite, load_graphs],
            whole_plate_info,
            num_parallel,
            time,
            "edge_post_process",
            cpus=num_cpus,
            name_job=name_job,
            node="fat",
        )
        # break

In [10]:
folders = folders.sort_values(by="datetime")
folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,Temp,unique_id,datetime
783,/projects/0/einf914/data/20220506_2350_Plate03,1048,3,'Carrot','C2','001P100N100C','Y','-',20220506,25,...,False,True,True,True,True,"06.05.2022, 23:50:",20220506_2350_Plate03,NaN,1048_20220506,2022-05-06 23:50:00
552,/projects/0/einf914/data/20220507_0150_Plate03,1048,3,'Carrot','C2','001P100N100C','Y','-',20220506,25,...,False,True,True,True,True,"07.05.2022, 01:50:",20220507_0150_Plate03,NaN,1048_20220506,2022-05-07 01:50:00
3050,/projects/0/einf914/data/20220507_0350_Plate03,1048,3,'Carrot','C2','001P100N100C','Y','-',20220506,25,...,False,True,True,True,True,"07.05.2022, 03:50:",20220507_0350_Plate03,NaN,1048_20220506,2022-05-07 03:50:00
3156,/projects/0/einf914/data/20220507_0550_Plate03,1048,3,'Carrot','C2','001P100N100C','Y','-',20220506,25,...,False,True,True,True,True,"07.05.2022, 05:50:",20220507_0550_Plate03,NaN,1048_20220506,2022-05-07 05:50:00
2150,/projects/0/einf914/data/20220507_0750_Plate03,1048,3,'Carrot','C2','001P100N100C','Y','-',20220506,25,...,False,True,True,True,True,"07.05.2022, 07:50:",20220507_0750_Plate03,NaN,1048_20220506,2022-05-07 07:50:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,/projects/0/einf914/data/20220619_1118_Plate30,1048,30,'Carrot','C2','001P100N100C','Y','-',20220506,25,...,False,True,True,True,True,"19.06.2022, 11:18:",20220619_1118_Plate30,NaN,1048_20220506,2022-06-19 11:18:00
827,/projects/0/einf914/data/20220619_1718_Plate30,1048,30,'Carrot','C2','001P100N100C','Y','-',20220506,25,...,False,True,True,True,True,"19.06.2022, 17:18:",20220619_1718_Plate30,NaN,1048_20220506,2022-06-19 17:18:00
939,/projects/0/einf914/data/20220619_2318_Plate30,1048,30,'Carrot','C2','001P100N100C','Y','-',20220506,25,...,False,True,True,True,True,"19.06.2022, 23:18:",20220619_2318_Plate30,NaN,1048_20220506,2022-06-19 23:18:00
1275,/projects/0/einf914/data/20220620_0518_Plate30,1048,30,'Carrot','C2','001P100N100C','Y','-',20220506,25,...,False,True,True,True,True,"20.06.2022, 05:18:",20220620_0518_Plate30,NaN,1048_20220506,2022-06-20 05:18:00


In [11]:
exp = Experiment(directory_targ)
exp.load(folders[:200:30])
exp.dates.sort()

/gpfs/home2/cbisot/pycode/AMFtrack/amftrack/pipeline/functions/image_processing/experiment_class_surf.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.folders["date"], format="%d.%m.%Y, %H:%M:"


2022-05-06 23:50:00
2022-05-09 15:39:00
2022-05-12 05:07:00
2022-05-14 17:52:00
2022-05-17 06:14:00
2022-05-20 10:38:00
2022-05-24 23:55:00


In [12]:
for t in range(exp.ts):
    exp.load_tile_information(t)

In [13]:
for t in range(exp.ts):
    exp.load_tile_information(t)

load_graphs(exp, directory_targ, post_process=True)

In [ ]:
t = 0
plot_edge_width(exp, t, lambda edge: edge.width(t), dilation=20)

In [15]:
import networkx as nx

resolution = 50
skip = False
for t in range(exp.ts)[:1]:
    (G, pos) = exp.nx_graph[t], exp.positions[t]
    edge_test = get_width_info_new(exp, t, resolution=resolution, skip=skip)
    nx.set_edge_attributes(G, edge_test, "width")
    exp.nx_graph[t] = G

True


In [16]:
save_graphs(exp)

2022-05-06 23:50:00 True
2022-05-09 15:39:00 True
2022-05-12 05:07:00 True
2022-05-14 17:52:00 True
2022-05-17 06:14:00 True
2022-05-20 10:38:00 True
2022-05-24 23:55:00 True


In [ ]:
t = 2
intervals = [[1, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 9]]

plot_edge_width(exp, t, lambda edge: edge.width(t), dilation=20, intervals=intervals)